# Data cleansing

## STEP 0: Initialization
Load config and initialize logging for a reproducible cleaning run.

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import logging

# Import centralized configuration + logger utility
sys.path.append(os.path.abspath('../src'))
import config
from utils import setup_logger

logger = setup_logger(
    name="data-cleansing",
    log_dir=getattr(config, 'LOG_DIR', os.path.join(os.getcwd(), 'log')),
    filename="data_cleansing.log",
    level=logging.INFO,
    mode='w',
)

2025-12-14 16:36:20,799 - INFO - Logging initialized. Saving logs to: /home/fustli/Documents/Uni/DeepL/End-of-trip_delay_prediction/log/data_cleansing.log


## STEP 1: Load raw dataset
Read the raw CSV and coerce `delay_seconds` to numeric for robust filtering.

In [2]:
raw_path = getattr(
    config,
    'RAW_CSV_PATH',
    os.path.join(getattr(config, 'DATA_DIR', os.path.join(os.getcwd(), 'data')), 'vehicle_positions.csv'),
)

if not os.path.exists(raw_path):
    raise FileNotFoundError(f"Data file not found at {raw_path}")

logger.info(f"Loading raw data from: {raw_path}")
df = pd.read_csv(raw_path)
df['delay_seconds'] = pd.to_numeric(df['delay_seconds'], errors='coerce')

initial_count = len(df)
logger.info(f"Initial data points: {initial_count:,}")

2025-12-14 16:36:20,806 - INFO - Loading raw data from: /home/fustli/Documents/Uni/DeepL/End-of-trip_delay_prediction/data/vehicle_positions.csv
2025-12-14 16:36:47,383 - INFO - Initial data points: 23,932,417


## STEP 2: Apply filters
Filter out missing/zero delays, unrealistic outliers, and GPS points outside the Budapest bounding box (all thresholds from config).

In [3]:
# --- 1. Define Filters ---

# A. Missing Data
mask_nan = df['delay_seconds'].isna()

# B. Exact Zero Delays (exclude NaNs to avoid double counting)
mask_zero = (df['delay_seconds'] == 0) & (~mask_nan)

# C. Unrealistic Delays
min_delay = int(getattr(config, 'MIN_REALISTIC_DELAY', -1800))
max_delay = int(getattr(config, 'MAX_REALISTIC_DELAY', 1800))
mask_delay_outliers = (
    (df['delay_seconds'] < min_delay) |
    (df['delay_seconds'] > max_delay)
)

# D. Geolocation Outliers (Budapest Bounding Box)
lat_min = float(getattr(config, 'LAT_MIN', 47.30))
lat_max = float(getattr(config, 'LAT_MAX', 47.65))
lon_min = float(getattr(config, 'LON_MIN', 18.90))
lon_max = float(getattr(config, 'LON_MAX', 19.35))
mask_geo_outliers = (
    (df['latitude'] < lat_min) | (df['latitude'] > lat_max) |
    (df['longitude'] < lon_min) | (df['longitude'] > lon_max)
)

# --- 2. Apply Filters ---
total_useless_mask = mask_nan | mask_zero | mask_delay_outliers | mask_geo_outliers
clean_df = df[~total_useless_mask].copy()
useless_df = df[total_useless_mask]

# --- 3. Report ---
logger.info(f"Removed {len(useless_df):,} unusable rows")
logger.info(f" - NaN delays:            {int(mask_nan.sum()):,}")
logger.info(f" - Zero delays:           {int(mask_zero.sum()):,}")
logger.info(f" - Delay outliers:        {int(mask_delay_outliers.sum()):,} (bounds {min_delay}..{max_delay})")
logger.info(f" - Geographical outliers: {int(mask_geo_outliers.sum()):,} (lat {lat_min}..{lat_max}, lon {lon_min}..{lon_max})")
logger.info(f"Remaining data points:    {len(clean_df):,}")

2025-12-14 16:36:49,586 - INFO - Removed 8,867,001 unusable rows
2025-12-14 16:36:49,595 - INFO -  - NaN delays:            8,621,295
2025-12-14 16:36:49,605 - INFO -  - Zero delays:           172,960
2025-12-14 16:36:49,614 - INFO -  - Delay outliers:        29,405 (bounds -1800..1800)
2025-12-14 16:36:49,623 - INFO -  - Geographical outliers: 1,143,669 (lat 47.3..47.65, lon 18.9..19.35)
2025-12-14 16:36:49,623 - INFO - Remaining data points:    15,065,416


## STEP 3: Save cleaned dataset
Write the cleaned CSV to `CLEANED_CSV_PATH` for downstream notebooks/scripts.

In [4]:
clean_path = getattr(
    config,
    'CLEANED_CSV_PATH',
    os.path.join(getattr(config, 'DATA_DIR', os.path.join(os.getcwd(), 'data')), 'vehicle_positions_cleaned.csv'),
)

logger.info(f"Saving cleaned data to: {clean_path}")
os.makedirs(os.path.dirname(clean_path), exist_ok=True)
clean_df.to_csv(clean_path, index=False)
logger.info("File saved successfully.")

2025-12-14 16:36:49,629 - INFO - Saving cleaned data to: /home/fustli/Documents/Uni/DeepL/End-of-trip_delay_prediction/data/vehicle_positions_cleaned.csv
2025-12-14 16:37:34,024 - INFO - File saved successfully.
